In [3]:
import requests
from bs4 import BeautifulSoup as bs, Tag
import re
import pandas as pd
import time

In [5]:
url = 'https://en.wikipedia.org/wiki/Forbes_Global_2000#2023'
page = requests.get(url)
response = requests.get(url)
html = response.text
status = response.status_code
if status == 200:
    page = response.text
    soup = bs(page)
else:
    print(f"Oops! Received status code {status}")

In [11]:
table = soup.find_all('table')[7]

In [15]:
headings = table.find_all('th')


In [17]:
headings = [title.text.strip() for title in headings]
headings


['Sector',
 'Company',
 'Headquarters',
 'Overall  rank',
 'Sales  (billion $)',
 'Profits  (billion $)',
 'Assets  (billion $)',
 'Market value  (billion $)']

In [19]:
df = pd.DataFrame(columns = headings)
df

,Sector,Company,Headquarters,Overall rank,Sales (billion $),Profits (billion $),Assets (billion $),Market value (billion $)


In [31]:
data_row = table.find_all('tr')


In [45]:
def clean_citations(text):
    return re.sub(r'\[\d+\]', '', text).strip()

cleaned_data = []

for row in data_row[1:]:
    if isinstance(row, str):
        parsed = bs(row, 'html.parser')
    elif isinstance(row, Tag):
        parsed = row
    else:
        continue

    row_data = parsed.find_all('td')
    table_rows = [clean_citations(data.text) for data in row_data]
    cleaned_data.append(table_rows)

print(cleaned_data)

[['Advertising', 'WPP', 'London', '0301#301', '18.7', '1.8', '42.4', '30.1'], ['Aerospace & Defense', 'Boeing', 'Chicago', '0076#76', '96.1', '5.2', '94.4', '85.3'], ['Air Courier', 'United Parcel Service', 'Sandy Springs', '0146#146', '58.1', '4.8', '38.3', '93.3'], ['Airline', 'Delta Air Lines', 'Atlanta', '0170#170', '40.5', '4.7', '53.3', '34.4'], ['Aluminum', 'Alcoa', 'New York City', '0757#757', '21.7', '-0.3', '36.1', '13.7'], ['Apparel/Accessories', 'Christian Dior', 'Paris', '0216#216', '41.6', '1.7', '68.1', '31.4'], ['Apparel/Footwear Retail', 'Inditex', 'A Coruña', '0310#310', '23.1', '3.2', '18.8', '103.2'], ['Auto & Truck Manufacturers', 'Toyota Motor', 'Toyota', '0010#10', '235.8', '19.3', '406.7', '177.0'], ['Auto & Truck Parts', 'Continental', 'Hanover', '0209#209', '43.5', '3.0', '36.7', '43.7'], ['Beverages', 'Anheuser-Busch InBev', 'Leuven', '0056#56', '43.6', '8.3', '136.5', '204.6'], ['Biotechs', 'Gilead Sciences', 'Foster City', '0118#118', '32.2', '18.1', '51.8'

In [51]:
for row in cleaned_data:
    df.loc[len(df)] = row
  

In [53]:
df

,Sector,Company,Headquarters,Overall rank,Sales (billion $),Profits (billion $),Assets (billion $),Market value (billion $)
0,Advertising,WPP,London,0301#301,18.7,1.8,42.4,30.1
1,Aerospace & Defense,Boeing,Chicago,0076#76,96.1,5.2,94.4,85.3
2,Air Courier,United Parcel Service,Sandy Springs,0146#146,58.1,4.8,38.3,93.3
3,Airline,Delta Air Lines,Atlanta,0170#170,40.5,4.7,53.3,34.4
4,Aluminum,Alcoa,New York City,0757#757,21.7,-0.3,36.1,13.7
...,...,...,...,...,...,...,...,...
238,Telecommunications services,AT&T,Dallas,0012#12,146.8,13.2,402.7,234.2
239,Thrifts & Mortgage Financing,Aareal Bank,Wiesbaden,1381#1381,1.4,0.4,57.2,2.1
240,Tobacco,British American Tobacco,London,0177#177,20.0,6.6,46.5,111.0
241,Trading Companies,Mitsubishi Corp,Tokyo,0136#136,59.8,2.7,135.7,27.7


In [59]:
#to solve KeyError: 'Overall rank'
df.rename(columns={df.columns[3]: 'Overall rank'}, inplace=True)

# Clean the rank values
df['Overall rank'] = df['Overall rank'].apply(lambda x: '#' + x.split('#')[-1] if isinstance(x, str) and '#' in x else x)

In [61]:
df


,Sector,Company,Headquarters,Overall rank,Sales (billion $),Profits (billion $),Assets (billion $),Market value (billion $)
0,Advertising,WPP,London,#301,18.7,1.8,42.4,30.1
1,Aerospace & Defense,Boeing,Chicago,#76,96.1,5.2,94.4,85.3
2,Air Courier,United Parcel Service,Sandy Springs,#146,58.1,4.8,38.3,93.3
3,Airline,Delta Air Lines,Atlanta,#170,40.5,4.7,53.3,34.4
4,Aluminum,Alcoa,New York City,#757,21.7,-0.3,36.1,13.7
...,...,...,...,...,...,...,...,...
238,Telecommunications services,AT&T,Dallas,#12,146.8,13.2,402.7,234.2
239,Thrifts & Mortgage Financing,Aareal Bank,Wiesbaden,#1381,1.4,0.4,57.2,2.1
240,Tobacco,British American Tobacco,London,#177,20.0,6.6,46.5,111.0
241,Trading Companies,Mitsubishi Corp,Tokyo,#136,59.8,2.7,135.7,27.7


In [63]:
df.to_csv(r'C:\Users\pmush\OneDrive\Desktop\Data Science Prog\webscrapping\industrylist.csv', index = False)